### aim: 

### date: 

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [3]:
import numpy as np
from cmocean import cm
import cartopy as cp
import cartopy.crs as ccrs
import netCDF4 as nc
import matplotlib.pyplot as plt
import xarray as xr
import sys
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE')
#list of models
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/UTILS')
import lom
import utils as ut

%matplotlib inline
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import cartopy.feature as cfeature
from importlib import reload
import matplotlib.path as mpath
import glob
import pickle
import pandas as pd
import seawater
import time
plt.rcParams.update({'font.size': 12})
font = {'family' : 'normal',
'weight' : 'normal',
'size'   : 12}

plt.rc('font', **font)

### data structure for boe

In [4]:

test = True

if test:
    
    #### gradients in BGC
    grad_dDICdz = 0.28 # (umol/L/m) average between 0-100m depth and 100-200m
    grad_DIC_surfdeep = 120 #umol/L between surface and 1000m depth
    grad_dFedz = 0.0007 #(nmol/L/m) (nmol/L/m)
    grad_dFev = 0.4 # deep minus surface ocean (nmol/L) between surface and 1000m depth (based on Tagliabue 2012 database)

    ### mean ecosystem parameters
    mu_L = 1 # 0-1 depending on month  (Light limitation; no units)
    dmudFe = 0.59 # (1/d/(nmolFe/L)) see worksheet Fer parameters "C-estimations-v2"
    diatconc = 1 # mean diatom concentration (umol/L)

    ### carbonate system variables
    mn_kg = 0.065 # (mol/m2/yr/ppm) xx-TJ I don't know what this is
    mn_SST = 2 #(oC)
    mn_Revfact = 14 # Definition: R=(dCO2/CO2)/(dDIC/DIC)
    mn_DIC = 2120 # (umol/L)
    mn_TA = 2290 # (ueq/L)
    mn_pCO2 = 380 # (ppm)


    yr = 1950
    tdir = '/gpfs/home/mep22dku/scratch/BOE-SOcarbon/data/'
    nam = f'{tdir}data-CLQdummy.nc'
    
    data_vars = {


            ### bgc gradients
        'grad_dDICdz': (['period','season','scenario'], np.zeros([3,5,2]),
                {'units': ''}),
        'grad_DIC_surfdeep': (['period','season','scenario'], np.zeros([3,5,2]),
                {'units': ''}),
        'grad_dFedz': (['period','season','scenario'], np.zeros([3,5,2]),
                {'units': ''}),
        'grad_dFev': (['period','season','scenario'], np.zeros([3,5,2]),
                {'units': ''}),

            ### bgc gradients
        'mu_L': (['period','season','scenario'], np.zeros([3,5,2]),
                {'units': ''}),
        'dmudFe': (['period','season','scenario'], np.zeros([3,5,2]),
                {'units': ''}),
        'diatconc': (['period','season','scenario'], np.zeros([3,5,2]),
                {'units': ''}),

            ### carbonate system variables
        'mn_kg': (['period','season','scenario'], np.zeros([3,5,2]),
                {'units': ''}),
        'mn_SST': (['period','season','scenario'], np.zeros([3,5,2]),
                {'units': ''}),
        'mn_Revfact': (['period','season','scenario'], np.zeros([3,5,2]),
                {'units': ''}),
        'mn_DIC': (['period','season','scenario'], np.zeros([3,5,2]),
                {'units': ''}),
        'mn_TA': (['period','season','scenario'], np.zeros([3,5,2]),
                {'units': ''}),
        'mn_pCO2': (['period','season','scenario'], np.zeros([3,5,2]),
                {'units': ''}),


    }

    coords = {'period': (['period'], ['1950-2000','2000-2050','2050-2100']),
              'season': (['season'], ['FY','DJF','MAM','JJA','SON']),
              'scenario': (['scenario'], ['ssp1','ssp3']),
             }
    
    
    # define global attributes
    attrs = {'made in':'BOE-SO-carbon/sketch.ipynb',
    'desc': ''
    }
    ds = xr.Dataset(data_vars=data_vars,
    coords=coords,
    attrs=attrs)
# ds.to_netcdf(savenam)

In [5]:
ds

<xarray.Dataset>
Dimensions:            (period: 3, scenario: 2, season: 5)
Coordinates:
  * period             (period) <U9 '1950-2000' '2000-2050' '2050-2100'
  * season             (season) <U3 'FY' 'DJF' 'MAM' 'JJA' 'SON'
  * scenario           (scenario) <U4 'ssp1' 'ssp3'
Data variables: (12/13)
    grad_dDICdz        (period, season, scenario) float64 0.0 0.0 ... 0.0 0.0
    grad_DIC_surfdeep  (period, season, scenario) float64 0.0 0.0 ... 0.0 0.0
    grad_dFedz         (period, season, scenario) float64 0.0 0.0 ... 0.0 0.0
    grad_dFev          (period, season, scenario) float64 0.0 0.0 ... 0.0 0.0
    mu_L               (period, season, scenario) float64 0.0 0.0 ... 0.0 0.0
    dmudFe             (period, season, scenario) float64 0.0 0.0 ... 0.0 0.0
    ...                 ...
    mn_kg              (period, season, scenario) float64 0.0 0.0 ... 0.0 0.0
    mn_SST             (period, season, scenario) float64 0.0 0.0 ... 0.0 0.0
    mn_Revfact         (period, season, scenario) float64 0.0 0.0 ... 0.0 0.0
    mn_DIC             (period, season, scenario) float64 0.0 0.0 ... 0.0 0.0
    mn_TA              (period, season, scenario) float64 0.0 0.0 ... 0.0 0.0
    mn_pCO2            (period, season, scenario) float64 0.0 0.0 ... 0.0 0.0
Attributes:
    made in:  BOE-SO-carbon/sketch.ipynb
    desc: